In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import tkinter as tk
from tkinter import messagebox as mb
from matplotlib import pyplot as plt
import scipy.stats
import statsmodels.stats.api as sms
import math
import sys
from PIL import ImageEnhance, Image

In [ ]:
folder = input('Which folder are your images located in? ')
os.chdir('C:\\Users\\abhim\\Desktop\\'+folder)
print('All Set!')

In [ ]:
whichDataset = input("Which Dataset do you want to choose a sample image from? ")
whichImage = input("Which Image do you want to use as the sample image? ")

In [ ]:
global image, Satisfied, alpha_mult, finalAlpha, beta, beta_sub
Satisfied = "Gibberish"
alpha_mult = np.array([1.0])
finalAlpha = 1.0
beta = 0.0
beta_sub = np.array([0.0])

redChannel = cv2.imread('r'+whichDataset+'_'+whichImage+'.tif')
greenChannel = cv2.imread('g'+whichDataset+'_'+whichImage+'.tif')
#cv2.divide(greenChannel, 3.6, greenChannel)

plt.rcParams['figure.figsize'] = [12, 8]
image = cv2.cvtColor(redChannel+greenChannel, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()
Satisfied = input("Are you satisfied with the contrast of the image? Y/N (case sensitive)): ")

while Satisfied == "N":
    alpha = float(input("What would you like to multiply the contrast by? (1.0 - 4.5)): "))
    finalAlpha *= alpha
    beta = 0 - finalAlpha*1.5
    alpha_mult = np.array([alpha])
    beta_sub = np.array([beta])
    cv2.multiply(image, alpha_mult, image)
    cv2.add(image, beta_sub, image)
    image = Image.fromarray(image)
    enhancer = ImageEnhance.Sharpness(image)
    factor = 3.6
    image = enhancer.enhance(factor)
    image = np.asarray(image)
    cv2.add(image, beta_sub, image)
    image = cv2.bilateralFilter(image,9,75,75)
    plt.imshow(image)
    plt.show()
    Satisfied = input("Are you satisfied with the contrast of the image? Y/N (caps sensitive)): ")

if Satisfied == "Y":
    print('Great!')
    print('Final Multiplier: ', finalAlpha)
    print('Final Subtraction: ', beta)
    print('Sharpness Factor: ', factor)
    alpha_mult = np.array([finalAlpha])
    beta_sub = np.array([beta])
    
else:
    sys.exit('Restart Please~!')

In [ ]:
index=["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('C:\\Users\\abhim\\Downloads\\colors.csv', names=index, header=None)
img = redChannel+greenChannel
cv2.multiply(img, alpha_mult, img)
cv2.add(img, beta_sub, img)
img = Image.fromarray(img)
enhancer = ImageEnhance.Sharpness(img)
factor = 3.6
img = enhancer.enhance(factor)
img = np.asarray(img)
cv2.add(image, beta_sub, image)
img = cv2.bilateralFilter(img,9,75,75)
cv2.namedWindow("Color Recognition App", cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
cv2.resizeWindow("Color Recognition App", 800, 550) 

In [ ]:
image2 = cv2.imread('g'+whichDataset+'_'+whichImage+'.tif',0)
plt.imshow(image2, cmap = 'jet'), plt.colorbar(fraction=0.05, pad=0.04)
plt.show()

In [ ]:
clicked = False
r = g = b = xpos = ypos = 0
livecellcount = 0
earlyapoptoticcount = 0
lateapoptoticcount = 0
necroticcount = 0

In [ ]:
def recognize_color(R,G,B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"]))
        if(d<=minimum):
            minimum = d
            cname = csv.loc[i,"color_name"]            
    return cname

In [ ]:
def mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b,g,r,xpos,ypos, clicked
        clicked = True
        xpos = x
        ypos = y
        b,g,r = img[y,x]
        b = int(b)
        g = int(g)
        r = int(r)

In [ ]:
global theNamesList
theNamesList = ["Live", "Early Apoptotic", "Late Apoptotic", "Necrotic"]

In [ ]:
def startcolorchk():
    global askarray, livecells, earlyapoptotic, lateapoptotic, necrotic
    livecells = False
    earlyapoptotic = False
    lateapoptotic = False 
    necrotic = False
    askarray = [livecells, earlyapoptotic, lateapoptotic, necrotic]
    for listVar in range(len(askarray)):
        if mb.askyesno(theNamesList[listVar]+' Cells', 'Are there '+theNamesList[listVar]+' Cells?'):
            mb.showinfo('Yes', theNamesList[listVar]+' Cell count will be initiated')
            askarray[listVar] = True
        else:
            mb.showinfo('No', theNamesList[listVar]+' Cell count will NOT be initiated')
            askarray[listVar] = False
tk.Button(text='Start\n\nREAD CAREFULLY:\n1) Be sure to read each prompt carefully before answering.\n2) After you are done answering all the prompts, you will have to close this dialog box to proceed and see the image.\n3) Click cells within a group that have a diverse intensity range to get the best training efficacy for your counts.', command=startcolorchk).pack(fill=tk.X)
tk.mainloop()

In [ ]:
cv2.namedWindow('Color Recognition App')

In [ ]:
cv2.setMouseCallback('Color Recognition App', mouse_click)

In [ ]:
global livecellarray, earlyapoptoticarray, lateapoptoticarray, necroticarray
global theArrayList, theBoolList, theBooleanList, theCellCountList

livecellsboolean = askarray[0]
earlyapoptoticboolean = askarray[1]
lateapoptoticboolean = askarray[2]
necroticboolean = askarray[3]

livecellarray = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
earlyapoptoticarray = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
lateapoptoticarray = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
necroticarray = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

theArrayList = [livecellarray, earlyapoptoticarray, lateapoptoticarray, necroticarray]
theBoolList = [askarray[0], askarray[1], askarray[2], askarray[3]]
theBooleanList = [livecellsboolean, earlyapoptoticboolean, lateapoptoticboolean, necroticboolean]
theCellCountList = [livecellcount, earlyapoptoticcount, lateapoptoticcount, necroticcount]

###############################################################################

for listVar in range(len(theArrayList)):
    if(theBooleanList[listVar]):
        root = tk.Tk()
        root.title(theNamesList[listVar] + " Cells")
        label = tk.Label(root, text="Double Click only the CENTER of NUCLEI of 10 " +  theNamesList[listVar] + " Cells")
        label.pack(side="top", fill="both", expand=True, padx=20, pady=20)
        button = tk.Button(root, text="OK", command=lambda: root.destroy())
        button.pack(side="bottom", fill="none", expand=True)
        root.mainloop()
        theBooleanList[listVar] = False
    
    while(theBoolList[listVar] and theCellCountList[listVar] <= 9):  
        cv2.imshow("Color Recognition App",img)  
        if (clicked):
#             #cv2.rectangle(image, startpoint , endpoint, color, thickness)-1 fills entire rectangle 
#             cv2.rectangle(img,(20,20), (750,60), (b,g,r), -1)
#             #Creating text string to display( Color nameand RGB values )
#             text = recognize_color(r,g,b) + ' R='+ str(r) +  ' G='+ str(g) +  ' B='+ str(b)
            theArrayList[listVar][theCellCountList[listVar]][0] = r
            theArrayList[listVar][theCellCountList[listVar]][1] = g
            theArrayList[listVar][theCellCountList[listVar]][2] = b
            theCellCountList[listVar] += 1

#             #cv2.putText(img,text,start,font(0-7),fontScale,color,thickness,lineType )
#             cv2.putText(img, text,(50,50),2,0.8,(255,255,255),2,cv2.LINE_AA)
#            # For very light colours we will display text in black colour
#             if(r+g+b>=600):
#                 cv2.putText(img, text,(50,50),2,0.8,(0,0,0),2,cv2.LINE_AA)
            
            clicked=False
        if cv2.waitKey(20) & 0xFF ==27:
            break
cv2.destroyAllWindows()

In [ ]:
#################################### DATA ##################################################
global livecellmean, earlyapoptoticmean, lateapoptoticmean, necroticmean
global live95, early95, late95, nec95
global theMeanList, the95List, rgbRangeList, colorList

livecellmean = earlyapoptoticmean = lateapoptoticmean = necroticmean = live95 = early95 = late95 = nec95 = [0,0,0]
theMeanList = [livecellmean, earlyapoptoticmean, lateapoptoticmean, necroticmean]
the95List = [live95, early95, late95, nec95]
colorList = ['red', 'green', 'blue']

for listVar in range(len(theMeanList)):
    theMeanList[listVar] = np.around(theArrayList[listVar].mean(axis=0)).astype(int)
    the95List[listVar] = np.around(np.array(sms.DescrStatsW(theArrayList[listVar]).tconfint_mean())).astype(int)
    the95List[listVar][the95List[listVar] < 0] = 0

liverange_red = liverange_green = liverange_blue = earlyrange_red = earlyrange_green = earlyrange_blue = laterange_red = laterange_green = laterange_blue = necrange_red = necrange_green = necrange_blue = [0,0]
rgbRangeList = [[liverange_red, liverange_green, liverange_blue], [earlyrange_red, earlyrange_green, earlyrange_blue], [laterange_red, laterange_green, laterange_blue], [necrange_red, necrange_green, necrange_blue]]
fig, ax = plt.subplots(4, 3)
xlim = [1,2,3,4,5,6,7,8,9,10]

for listVar in range(len(rgbRangeList)):
    for nestListVar in range(len(rgbRangeList[0])):
        rgbRangeList[listVar][nestListVar] = [np.amin(theArrayList[listVar][:,nestListVar]), np.amax(theArrayList[listVar][:,nestListVar])]
        ax[listVar, nestListVar].bar(xlim, theArrayList[listVar][:,nestListVar], color = colorList[nestListVar], linewidth=1.2, width = 0.5)
        ax[listVar, nestListVar].set_xlim([0,11])
        ax[listVar, nestListVar].set_ylim([0,255])

for listVar in range(len(rgbRangeList)):
    ax[listVar,1].title.set_text(theNamesList[listVar] + " Cells")
    ax[listVar,0].set_ylabel('R/G/B Values')

fig.tight_layout()

In [ ]:
###################################### STATS ##################################
# stats f_oneway functions takes the groups as input and returns F and P-value
fvalue, pvalue = scipy.stats.f_oneway(theArrayList[0],
                                theArrayList[1], 
                                theArrayList[2],
                                theArrayList[3])
print(f"Results of ANOVA test:\n The F-statistic is: {fvalue}\n The p-value is: {pvalue}")

In [ ]:
t2, p2 = scipy.stats.ttest_ind(theArrayList[2],theArrayList[3])
print(f"Results of t-test:\n The t-statistic is: {t2}\n The p-value is: {p2}")
if (p2[0] > 0.05 and p2[1] > 0.05) and (theMeanList[3][1] > 15*finalAlpha or theMeanList[3][1]/theMeanList[0][1] > 0.15*finalAlpha):
    print('\nNOTE: There are NO necrotic cells! All presumed necrotic cells are late apoptotic! Necrotic values should be set to 0\'s.\nMerge the RGB ranges of Late Apoptotic and Necrotic.\nIgnore the Cell parameters for necrotic cells.')
if (p2[0] > 0.05 and p2[1] > 0.05) and (theMeanList[3][1] <= 15*finalAlpha or theMeanList[3][1]/theMeanList[0][1] <= 0.15*finalAlpha):
    print('\nNOTE: There are NO late apoptotic cells! All presumed apoptotic cells are necrotic! Late apoptotic values should be set to 0\'s.\nMerge the RGB ranges of Late Apoptotic and Necrotic.\nIgnore the Cell parameters for late apoptotic cells.')

In [ ]:
original = image.copy()

In [ ]:
print("RGB Mean (given as [R  G  B])\n")

global colorArray, lowerColorArray, upperColorArray, meanSizeList, minSizeList, conSizeList
colorArray = np.array([[[0,0,0],[0,0,0]],[[0,0,0],[0,0,0]],[[0,0,0],[0,0,0]],[[0,0,0],[0,0,0]]])
lowerColorArray = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
upperColorArray = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
minCellMult = [1,1,1,1]
minCellReq = [0.,0.,0.,0.]
conCellMult = [1,1,1,1]
meanCellMult = [1.,1.,1.,1.]
meanSizeList = [0,0,0,0]
minSizeList = [0,0,0,0]
conSizeList = [0,0,0,0]
color_outline = [[0,255,0],[255,255,0],[205,125,0],[255,0,0]]


for listVar in range(len(colorArray)):
    for nestListVar in range(len(colorArray[0,0])):
        for doublenestListVar in range(len(colorArray[0])):
            colorArray[listVar, doublenestListVar, nestListVar] = np.array([rgbRangeList[listVar][nestListVar][doublenestListVar]])

for listVar in range(len(colorArray)):
    lowerColorArray[listVar] = colorArray[listVar,0]
    upperColorArray[listVar] = colorArray[listVar,1]
    mask = cv2.inRange(original, lowerColorArray[listVar], upperColorArray[listVar]) #  cv2.Canny(image_max,95,95) #
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=0) 
    close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        cv2.drawContours(original, [c], -1, color_outline[listVar], 6) #36,255,12
            
lowerColorArray2 = np.array([[0,250,0],[250,250,0],[200,120,0],[250,0,0]])
upperColorArray2 = np.array([[1,255,1],[255,255,1],[205,125,1],[255,1,0]])
        
for listVar in range(len(colorArray)):
    mask = cv2.inRange(original, lowerColorArray2[listVar], upperColorArray2[listVar]) #  cv2.Canny(image_max,95,95) #
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=0) 
    close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    all_areas = []
    for c in cnts:
        area = cv2.contourArea(c)
        all_areas.append(area)

    if upperColorArray[listVar][1] == 0:
        print('Nothing to show... carry on!\n')
    else:
        meanSizeList[listVar] = np.around(np.mean(all_areas)*meanCellMult[listVar])
        minSizeList[listVar] = np.around(min(all_areas)*minCellMult[listVar]+minCellReq[listVar])
        conSizeList[listVar] = np.around(np.mean(all_areas)*conCellMult[listVar])
        print(theNamesList[listVar],"Cells MEAN: ", theMeanList[listVar], "\nR: ", "RANGE: ", rgbRangeList[listVar][0], ", 95% CI: ", the95List[listVar][:,0], "\nG: ", "RANGE: ", rgbRangeList[listVar][1], ", 95% CI: ", the95List[listVar][:,1], "\nB: ", "RANGE: ", rgbRangeList[listVar][2], ", 95% CI: ", the95List[listVar][:,2], "\n")
        print(theNamesList[listVar],"Cell Parameters: \nAverage Cell Area: ", meanSizeList[listVar], "\nMinimum Cell Area: ", minSizeList[listVar], "\nConnected Cell Area: ", conSizeList[listVar], "\n---------------------------\n")

In [ ]:
%store lowerColorArray
%store upperColorArray
%store meanSizeList
%store minSizeList
%store conSizeList
%store alpha_mult
%store finalAlpha
%store beta_sub
%store mean_inten

In [ ]:
import datetime
with open("C:\\Users\\abhim\\Desktop\\RainbowCountsVars.txt", mode='a') as file:
    file.write('-------------------------------\n')
    file.write('%s\n' % (datetime.datetime.now()))
    file.write('Lower Color Array: %s\n' % (lowerColorArray))
    file.write('Upper Color Array: %s\n' % (upperColorArray))
    file.write('Mean Size List: %s\n' % (meanSizeList))
    file.write('Min Size List: %s\n' % (minSizeList))
    file.write('Connected Size List: %s\n' % (conSizeList))
    file.write('alpha_mult: %s\n' % (alpha_mult))
    file.write('final Alpha: %s\n' % (finalAlpha))
    file.write('beta_sub: %s\n' % (beta_sub))
    file.write('Mean Intensity: %s\n\n' % (mean_inten))

In [ ]:
%tb